In [1]:
import pandas as pd

In [9]:
seattle_calendar_df = pd.read_csv("data/airbnb/seattle/calendar.csv")
seattle_calendar_df.head()
seattle_calendar_df.describe()

,listing_id
count,1.393570e+06
mean,5.550111e+06
std,2.962274e+06
min,3.335000e+03
25%,3.258213e+06
50%,6.118244e+06
75%,8.035212e+06
max,1.034016e+07
